In [1]:
import numpy as np
import os 
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torch import nn
import torchsummary
from torch.utils.data import DataLoader
from collections import defaultdict
from torchvision.utils import make_grid

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Utils

In [3]:
CLASS_NAME_TO_ID = {'stray': 0, 'target': 1}
CLASS_ID_TO_NAME = {0: 'stray', 1: 'target'}
BOX_COLOR = {'stray':(200, 0, 0), 'target':(0, 0, 200)}
TEXT_COLOR = (255, 255, 255)

def save_model(model_state, model_name, save_dir="./trained_model"):
    os.makedirs(save_dir, exist_ok=True)
    torch.save(model_state, os.path.join(save_dir, model_name))


def visualize_bbox(image, bbox, class_name, color=BOX_COLOR, thickness=2):
    x_center, y_center, w, h = bbox
    x_min = int(x_center - w/2)
    y_min = int(y_center - h/2)
    x_max = int(x_center + w/2)
    y_max = int(y_center + h/2)
    
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=color[class_name], thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(image, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), color[class_name], -1)
    cv2.putText(
        image,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return image


def visualize(image, bboxes, category_ids):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
#         print('category_id: ',category_id)
        class_name = CLASS_ID_TO_NAME[category_id.item()]
        img = visualize_bbox(img, bbox, class_name)
    return img

# Datasets

In [4]:
class Radio_dataset():
    def __init__(self, path, phase, transformer=None, aug=None, aug_factor=0):
        self.path=path
        self.phase=phase
        self.transformer=transformer
        self.aug=aug
        self.aug_factor=aug_factor
        
        self.image_files = sorted([fn for fn in os.listdir(self.path+"/"+self.phase+"/image") if fn.endswith("jpg")])
        self.label_files= sorted([lab for lab in os.listdir(self.path+"/"+self.phase+"/label") if lab.endswith("txt")])
        
        self.auged_img_list, self.auged_label_list=self.make_aug_list(self.image_files, self.label_files)
        
    def __getitem__(self,index):
        if(self.aug==None):
            filename, image = self.get_image(index)
            bboxes, class_ids = self.get_label(index)

            if(self.transformer):
                transformed_data=self.transformer(image=image, bboxes=bboxes, class_ids=class_ids)
                image = transformed_data['image']
                bboxes = np.array(transformed_data['bboxes'])
                class_ids = np.array(transformed_data['class_ids'])


            target = {}
    #         print(f'bboxes:{bboxes}\nclass_ids:{class_ids}\nlen_bboxes:{len(bboxes)}\nlen_class_ids:{len(class_ids)}')
    #         print(f'filename: {filename}')
            target["boxes"] = torch.Tensor(bboxes).float()
            target["labels"] = torch.Tensor(class_ids).long()

            ###
            bboxes=torch.Tensor(bboxes).float()
            class_ids=torch.Tensor(class_ids).long()
#             print(f"bboxes:{bboxes}\nclass_ids:{class_ids}\n---------")
            target = np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1)
            ###
        else:
            image=self.auged_img_list[index][1]
            target=self.auged_label_list[index]
            filename=self.auged_img_list[index][0]
        return image, target, filename
    
    def __len__(self, ):
        length=0
        if(self.aug==None):
            length=len(self.image_files)
        else:
            length=len(self.auged_img_list)
        return length
    
    def make_aug_list(self,ori_image_list,ori_label_files):
        aug_image_list=[]
        aug_label_list=[]
        
        print(f"start making augmented images-- augmented factor:{self.aug_factor}")
        for i in range(len(ori_image_list)):
            filename, ori_image = self.get_image(i)
            ori_bboxes, ori_class_ids = self.get_label(i)
            for j in range(self.aug_factor):
                auged_data=self.aug(image=ori_image, bboxes=ori_bboxes, class_ids=ori_class_ids)
                image = auged_data['image']
                bboxes = np.array(auged_data['bboxes'])
                class_ids = np.array(auged_data['class_ids'])
                
                bboxes=torch.Tensor(bboxes).float()
                class_ids=torch.Tensor(class_ids).long()
                
                aug_image_list.append((filename, image))
#                 print(f"{filename}, {bboxes}, {class_ids[:, np.newaxis]}")
                aug_label_list.append(np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1))
        
        print(f"total length of augmented images: {len(aug_image_list)}")
        
        return aug_image_list, aug_label_list
        
    
    def get_image(self, index): # 이미지 불러오는 함수
        filename = self.image_files[index]
        
        image_path = self.path+"/"+self.phase+"/image/"+filename
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return filename, image
    
    def get_label(self, index): # label (box좌표, class_id) 불러오는 함수
        label_filename=self.label_files[index]
        label_path = self.path+"/"+self.phase+"/label/"+label_filename
        with open(label_path, 'r') as file:
            labels = file.readlines()
        
        class_ids=[]
        bboxes=[]
        for label in labels:
            label=label.replace("\n", "")
            obj=label.split(' ')[0]
            coor=label.split(' ')[1:]
            obj=int(obj)
            coor=list(map(float, coor))
            class_ids.append(obj)
            bboxes.append(coor)
            
        return bboxes, class_ids
    

In [5]:
IMAGE_SIZE = 448

transformer = A.Compose([ 
        # bounding box의 변환, augmentation에서 albumentations는 Detection 학습을 할 때 굉장히 유용하다. 
        A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
        A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        # albumentations 라이브러리에서는 Normalization을 먼저 진행해 주고 tensor화를 진행해 주어야한다.
    ],
    # box 위치에 대한 transformation도 함께 진행된다. 
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
)

augmentator=A.Compose([
#     A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
    A.HorizontalFlip(p=0.7),
    A.VerticalFlip (p=0.5),
    A.HueSaturationValue(p=0.5),
    A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
)

def collate_fn(batch):
    image_list = []
    target_list = []
    filename_list = []
    
    for a,b,c in batch:
        image_list.append(a)
        target_list.append(b)
        filename_list.append(c)

    return torch.stack(image_list, dim=0), target_list, filename_list


In [6]:
# NECK_PATH = '/home/host_data/PET_data/Neck'
# BODY_PATH = '/home/host_data/PET_data/Body'

NECK_PATH = '/home/host_data/PET_data_IP_AUG/aug_patched_Neck/'
BODY_PATH = '/home/host_data/PET_data_image_patching/Body'

PATH='/home/host_data/radio_signal_data_organized/'

# trainset_yes_aug=PET_dataset(part='neck',neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='train', transformer=transformer, aug=augmentator, aug_factor=5)
trainset_no_aug=Radio_dataset(path=PATH, phase='valid', transformer=transformer, aug=augmentator, aug_factor=2)


start making augmented images-- augmented factor:2
total length of augmented images: 1130


In [18]:
len(trainset_no_aug)

1130

In [19]:
@interact(index=(0, len(trainset_no_aug)-1))

def show_sample(index=0):
    image, target, filename = trainset_no_aug[index]
    image=image.permute(1,2,0).numpy()
    img_H, img_W, _ = image.shape
    print(filename)
    print(image.shape)
#     print(image)

#     bboxes = target['boxes']
#     class_ids = target["labels"]
    
    ###
    bboxes = target[:, 0:4]
    class_ids = target[:, 4]
    ###
    bboxes[:, [0,2]] *= img_W
    bboxes[:, [1,3]] *= img_H

    canvas = visualize(image, bboxes, class_ids)
    plt.figure(figsize=(6,6))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

# show_sample()

interactive(children=(IntSlider(value=0, description='index', max=1129), Output()), _dom_classes=('widget-inte…

In [9]:
@interact(index=(0, len(trainset_yes_aug)-1))

def show_sample(index=0):
    image, target, filename = trainset_yes_aug[index]
    image=image.permute(1,2,0).numpy()
    img_H, img_W, _ = image.shape
    print(filename)
    print(image.shape)
#     print(image)

#     bboxes = target['boxes']
#     class_ids = target["labels"]
    ###
    bboxes = target[:, 0:4]
    class_ids = target[:, 4]
    ###
    bboxes[:, [0,2]] *= img_W
    bboxes[:, [1,3]] *= img_H
    print(bboxes)

    canvas = visualize(image, bboxes, class_ids)
    plt.figure(figsize=(6,6))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

# show_sample()

interactive(children=(IntSlider(value=0, description='index', max=1049), Output()), _dom_classes=('widget-inte…

## Model

In [6]:
class YOLO_SWIN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.num_classes = num_classes
        self.num_bboxes = 2
        self.grid_size = 7

#         resnet18 = torchvision.models.resnet18(pretrained = True)
        swin=torchvision.models.swin_v2_t(weights='IMAGENET1K_V1')
        layers = [m for m in swin.children()] #Resnet에서 Yolo에서 가져올수 있을만한 layer만 선별적으로 가져오기 위해서

        # 기존 Resnet18의 layer들중에서 맨 뒤에 두개만 제외하고 다 가져와서 Backbone으로 사용
        self.backbone = nn.Sequential(*layers[:-3]) 
        self.head = nn.Sequential(
                nn.Conv2d(in_channels=768, out_channels=1024, kernel_size=1, padding=0,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),

                nn.Conv2d(in_channels=1024, out_channels=(4+1)*self.num_bboxes+num_classes, kernel_size=1, padding=0, bias=False),
                nn.AdaptiveAvgPool2d(output_size=(self.grid_size, self.grid_size))
            )

    def forward(self, x):
        out = self.backbone(x)
        # out = self.neck(out)
        out = self.head(out) # input (batch, 3, 448, 448) -> output feature (batch, 12, 7, 7)
        return out


In [7]:
NUM_CLASSES = 2
model = YOLO_SWIN(num_classes=NUM_CLASSES)
model.to(device)

YOLO_SWIN(
  (backbone): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): Permute()
        (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (1): Sequential(
        (0): SwinTransformerBlockV2(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): ShiftedWindowAttentionV2(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (0): Linear(in_features=96, out_f

In [8]:
torchsummary.summary(model, (3,448,448))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 112, 112]           4,704
           Permute-2         [-1, 112, 112, 96]               0
         LayerNorm-3         [-1, 112, 112, 96]             192
            Linear-4          [-1, 15, 15, 512]           1,536
              ReLU-5          [-1, 15, 15, 512]               0
            Linear-6            [-1, 15, 15, 3]           1,536
ShiftedWindowAttentionV2-7         [-1, 112, 112, 96]               0
         LayerNorm-8         [-1, 112, 112, 96]             192
   StochasticDepth-9         [-1, 112, 112, 96]               0
           Linear-10        [-1, 112, 112, 384]          37,248
             GELU-11        [-1, 112, 112, 384]               0
          Dropout-12        [-1, 112, 112, 384]               0
           Linear-13         [-1, 112, 112, 96]          36,960
          Dropout-14         [-1,

In [9]:
x = torch.randn(1, 3, 448, 448).to(device)
with torch.no_grad():
    y = model(x)
print(y.shape)

torch.Size([1, 12, 7, 7])


# Loss func

In [10]:
class YOLO_LOSS():
    def __init__(self, num_classes, device, lambda_coord=5., lambda_noobj=0.5):
        self.num_classes = num_classes
        self.device = device
        self.grid_size = 7
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj
        self.mse_loss = nn.MSELoss(reduction="sum")

    def __call__(self, predictions, targets):
        self.batch_size, _, _, _ = predictions.shape
        groundtruths = self.build_batch_target_grid(targets)
        groundtruths = groundtruths.to(self.device)
        
        with torch.no_grad():
            iou1 = self.get_IoU(predictions[:, 1:5, ...], groundtruths[:, 1:5, ...])
            iou2 = self.get_IoU(predictions[:, 6:10, ...], groundtruths[:, 1:5, ...])

        ious = torch.stack([iou1, iou2], dim=1)
        max_iou, best_box = ious.max(dim=1, keepdim=True)
        max_iou = torch.cat([max_iou, max_iou], dim=1)
        best_box = torch.cat([best_box.eq(0), best_box.eq(1)], dim=1)

        predictions_ = predictions[:, :5*2, ...].reshape(self.batch_size, 2, 5, self.grid_size, self.grid_size)
        obj_pred = predictions_[:, :, 0, ...]
        xy_pred = predictions_[:, :, 1:3, ...]
        wh_pred = predictions_[:, :, 3:5, ...]
        cls_pred = predictions[:, 5*2:, ...]

        groundtruths_ = groundtruths[:, :5, ...].reshape(self.batch_size, 1, 5, self.grid_size, self.grid_size)
        obj_target = groundtruths_[:, :, 0, ...]
        xy_target = groundtruths_[:, :, 1:3, ...]
        wh_target= groundtruths_[:, :, 3:5, ...]
        cls_target = groundtruths[:, 5:, ...]
        
        positive = obj_target * best_box

        obj_loss = self.mse_loss(positive * obj_pred, positive * ious)
        noobj_loss = self.mse_loss((1 - positive) * obj_pred, ious*0)
        xy_loss = self.mse_loss(positive.unsqueeze(dim=2) * xy_pred, positive.unsqueeze(dim=2) * xy_target)
        wh_loss = self.mse_loss(positive.unsqueeze(dim=2) * (wh_pred.sign() * (wh_pred.abs() + 1e-8).sqrt()),
                           positive.unsqueeze(dim=2) * (wh_target + 1e-8).sqrt())
        cls_loss = self.mse_loss(obj_target * cls_pred, cls_target)
        
        obj_loss /= self.batch_size
        noobj_loss /= self.batch_size
        bbox_loss = (xy_loss+wh_loss) / self.batch_size
        cls_loss /= self.batch_size
        
        total_loss = obj_loss + self.lambda_noobj*noobj_loss + self.lambda_coord*bbox_loss + cls_loss
        return total_loss, (obj_loss.item(), noobj_loss.item(), bbox_loss.item(), cls_loss.item())
    
    def build_target_grid(self, target):
        target_grid = torch.zeros((1+4+self.num_classes, self.grid_size, self.grid_size), device=self.device)

        for gt in target:
            xc, yc, w, h, cls_id = gt
            xn = (xc % (1/self.grid_size))
            yn = (yc % (1/self.grid_size))
            cls_id = int(cls_id)

            i_grid = int(xc * self.grid_size)
            j_grid = int(yc * self.grid_size)
            target_grid[0, j_grid, i_grid] = 1
            target_grid[1:5, j_grid, i_grid] = torch.Tensor([xn,yn,w,h])
#             print(5+cls_id, j_grid, i_grid)
            target_grid[5+cls_id, j_grid, i_grid] = 1

        return target_grid
    
    def build_batch_target_grid(self, targets):
        target_grid_batch = torch.stack([self.build_target_grid(target) for target in targets], dim=0)
        return target_grid_batch
    
    def get_IoU(self, cbox1, cbox2):
        box1 = self.xywh_to_xyxy(cbox1)
        box2 = self.xywh_to_xyxy(cbox2)

        x1 = torch.max(box1[:, 0, ...], box2[:, 0, ...])
        y1 = torch.max(box1[:, 1, ...], box2[:, 1, ...])
        x2 = torch.min(box1[:, 2, ...], box2[:, 2, ...])
        y2 = torch.min(box1[:, 3, ...], box2[:, 3, ...])

        intersection = (x2-x1).clamp(min=0) * (y2-y1).clamp(min=0)
        union = abs(cbox1[:, 2, ...]*cbox1[:, 3, ...]) + \
                abs(cbox2[:, 2, ...]*cbox2[:, 3, ...]) - intersection

        intersection[intersection.gt(0)] = intersection[intersection.gt(0)] / union[intersection.gt(0)]
        return intersection
    
    def generate_xy_normed_grid(self):
        y_offset, x_offset = torch.meshgrid(torch.arange(self.grid_size), torch.arange(self.grid_size))
        xy_grid = torch.stack([x_offset, y_offset], dim=0)
        xy_normed_grid = xy_grid / self.grid_size
        return xy_normed_grid.to(self.device)

    def xywh_to_xyxy(self, bboxes):
        xy_normed_grid = self.generate_xy_normed_grid()
        xcyc = bboxes[:,0:2,...] + xy_normed_grid.tile(self.batch_size, 1,1,1)
        wh = bboxes[:,2:4,...]
        x1y1 = xcyc - (wh/2)
        x2y2 = xcyc + (wh/2)
        return torch.cat([x1y1, x2y2], dim=1)

## Train

In [11]:
def train_one_epoch(dataloaders, model, criterion, optimizer, device):
    train_loss = defaultdict(float)
    val_loss = defaultdict(float)
    
    for phase in ["train", "val"]:
        if phase == "train":
            model.train()
        else:
            model.eval()
        
        running_loss = defaultdict(float)
        for index, batch in enumerate(dataloaders[phase]):
            images = batch[0].to(device)
            targets = batch[1]
            filenames = batch[2]
            
            with torch.set_grad_enabled(phase == "train"): # phase가 train 일때만 gradient 추적기능을 킨다.
                predictions = model(images) #prediction shape=> B,12,7,7
#             print(f"predictions:{predictions}, \ntargets: {targets}\n")
            loss, (obj_loss, noobj_loss, bbox_loss, cls_loss) = criterion(predictions, targets)
#             print(f"loss:{loss}, obj_loss:{obj_loss}, noobj_loss:{noobj_loss}\nbbox_loss:{bbox_loss}, cls_loss:{cls_loss}\n--------------\n")
            if phase == "train":
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # 현재 epoch단계에서 loss가 얼마인지 running loss 가출력
                running_loss["total_loss"] += loss.item()
                running_loss["obj_loss"] += obj_loss
                running_loss["noobj_loss"] += noobj_loss
                running_loss["bbox_loss"] += bbox_loss
                running_loss["cls_loss"] += cls_loss
                
                train_loss["total_loss"] += loss.item()
                train_loss["obj_loss"] += obj_loss
                train_loss["noobj_loss"] += noobj_loss
                train_loss["bbox_loss"] += bbox_loss
                train_loss["cls_loss"] += cls_loss
                
                if (index > 0) and (index % VERBOSE_FREQ) == 0:
                    text = f"<<<iteration:[{index}/{len(dataloaders[phase])}] - "
                    for k, v in running_loss.items():
                        text += f"{k}: {v/VERBOSE_FREQ:.4f}  "
                        running_loss[k] = 0.
                    print(text)
            else:
                val_loss["total_loss"] += loss.item()
                val_loss["obj_loss"] += obj_loss
                val_loss["noobj_loss"] += noobj_loss
                val_loss["bbox_loss"] += bbox_loss
                val_loss["cls_loss"] += cls_loss

    for k in train_loss.keys():
        train_loss[k] /= len(dataloaders["train"])
        val_loss[k] /= len(dataloaders["val"])
    return train_loss, val_loss

In [12]:
def build_dataloader(PATH, batch_size=2, aug_factor=0):
    IMAGE_SIZE = 448
    transformer = A.Compose([
            A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
            A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
    )
    augmentator=A.Compose([
        A.HorizontalFlip(p=0.7),
        A.VerticalFlip (p=0.5),
        A.HueSaturationValue(p=0.5),
        A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
        A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
    )
    
#     Radio_dataset(path=PATH, phase='train', transformer=transformer, aug=None)
#     trainset_no_aug=Radio_dataset(path=PATH, phase='train', transformer=transformer, aug=augmentator, aug_factor=2)
    dataloaders = {}
    train_dataset=Radio_dataset(path=PATH, phase='train', transformer=transformer, aug=augmentator, aug_factor=2)
    dataloaders["train"] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    val_dataset=Radio_dataset(path=PATH, phase='valid', transformer=transformer, aug=augmentator, aug_factor=2)
    dataloaders["val"] = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    print(f"trainset:{len(train_dataset)} validset:{len(val_dataset)}")
    return dataloaders

In [13]:
# data_dir = "/content/drive/MyDrive/fastCamMedicalProj/DATASET/DATASET/Detection/"
# NECK_PATH = '/home/host_data/PET_data/Neck'
# BODY_PATH = '/home/host_data/PET_data/Body'
# NECK_PATH = '/home/host_data/PET_data_IP_AUG/aug_patched_Neck'
# BODY_PATH = '/home/host_data/PET_data_IP_AUG/Body'

path='/home/host_data/radio_signal_data_organized/'
is_cuda = True

NUM_CLASSES = 2
IMAGE_SIZE = 448
BATCH_SIZE = 16
VERBOSE_FREQ = 20
LR=0.0001
AUG_FACTOR=4
PATCH_FACTOR=50
BACKBONE="YOLO_SWIN_T"
PART="RADIO"
num_epochs = 100
# DEVICE = torch.device('cuda' if torch.cuda.is_available and is_cuda else 'cpu')

dataloaders = build_dataloader(PATH=path,batch_size=BATCH_SIZE, aug_factor=AUG_FACTOR)
model = YOLO_SWIN(num_classes=NUM_CLASSES)
model = model.to(device)
criterion = YOLO_LOSS(num_classes=NUM_CLASSES, device=device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

start making augmented images-- augmented factor:2
0_target_8197cc8edd78543e0f9df05fe36d719d.jpg, tensor([[0.4739, 0.4653, 0.0228, 0.2708]]), tensor([[1]])
0_target_8197cc8edd78543e0f9df05fe36d719d.jpg, tensor([[0.5261, 0.5347, 0.0228, 0.2708]]), tensor([[1]])
1002_stray_3327a66a04cd703314beb6655dc45add.jpg, tensor([[0.5065, 0.7396, 0.0228, 0.1458]]), tensor([[0]])
1002_stray_3327a66a04cd703314beb6655dc45add.jpg, tensor([[0.5065, 0.7396, 0.0228, 0.1458]]), tensor([[0]])
1005_stray_cb9c983b1e8ee2d89dd1084924aea2f1.jpg, tensor([[0.5945, 0.3663, 0.0293, 0.1632]]), tensor([[0]])
1005_stray_cb9c983b1e8ee2d89dd1084924aea2f1.jpg, tensor([[0.4055, 0.6337, 0.0293, 0.1632]]), tensor([[0]])
1007_stray_c60e12b0bf36d502bab96ed9e1b2d922.jpg, tensor([[0.0570, 0.1128, 0.0489, 0.2257]]), tensor([[0]])
1007_stray_c60e12b0bf36d502bab96ed9e1b2d922.jpg, tensor([[0.9430, 0.8872, 0.0489, 0.2257]]), tensor([[0]])
1008_target_50baa9590398af8d65556720215a0351.jpg, tensor([[0.4951, 0.5069, 0.0326, 0.2847]]), ten

1055_target_3d785db001ab4b92788dbde93dd8bb17.jpg, tensor([[0.4951, 0.4861, 0.0326, 0.2014]]), tensor([[1]])
1056_target_47c0bf127600c5358178a5f70524604a.jpg, tensor([[0.5782, 0.5191, 0.0358, 0.2951]]), tensor([[1]])
1056_target_47c0bf127600c5358178a5f70524604a.jpg, tensor([[0.5782, 0.4809, 0.0358, 0.2951]]), tensor([[1]])
1059_stray_66237583ba21d860220cf57a529391b8.jpg, tensor([[0.3974, 0.9549, 0.0195, 0.0903]]), tensor([[0]])
1059_stray_66237583ba21d860220cf57a529391b8.jpg, tensor([[0.3974, 0.0451, 0.0195, 0.0903]]), tensor([[0]])
1060_stray_dfadc5bd4df6e9d9b0fe4580f8c171b0.jpg, tensor([[0.8388, 0.5243, 0.0358, 0.1389]]), tensor([[0]])
1060_stray_dfadc5bd4df6e9d9b0fe4580f8c171b0.jpg, tensor([[0.8388, 0.5243, 0.0358, 0.1389]]), tensor([[0]])
1062_target_4ffe3cfabb51d35c9b195584694fec86.jpg, tensor([[0.5000, 0.5174, 0.0228, 0.3403]]), tensor([[1]])
1062_target_4ffe3cfabb51d35c9b195584694fec86.jpg, tensor([[0.5000, 0.5174, 0.0228, 0.3403]]), tensor([[1]])
1065_target_537b7613b61f440d858e

1121_target_40861cca5e60edbcecbc984035cb08bc.jpg, tensor([[0.4984, 0.5087, 0.0261, 0.2882]]), tensor([[1]])
1121_target_40861cca5e60edbcecbc984035cb08bc.jpg, tensor([[0.4984, 0.4913, 0.0261, 0.2882]]), tensor([[1]])
1122_stray_7542458ad4248571861385987ed94eac.jpg, tensor([[0.6352, 0.4184, 0.0391, 0.2604]]), tensor([[0]])
1122_stray_7542458ad4248571861385987ed94eac.jpg, tensor([[0.6352, 0.4184, 0.0391, 0.2604]]), tensor([[0]])
1125_stray_f905e2cb3c063495990d8c79f4ad66a2.jpg, tensor([[0.5326, 0.9132, 0.0358, 0.1528]]), tensor([[0]])
1125_stray_f905e2cb3c063495990d8c79f4ad66a2.jpg, tensor([[0.5326, 0.0868, 0.0358, 0.1528]]), tensor([[0]])
1126_stray_75ef278b416f26e32027986e8050df75.jpg, tensor([[0.7606, 0.7674, 0.0293, 0.1806]]), tensor([[0]])
1126_stray_75ef278b416f26e32027986e8050df75.jpg, tensor([[0.7606, 0.7674, 0.0293, 0.1806]]), tensor([[0]])
1127_target_06b7e77a9cd5b65c7fc71ae59b2a29f5.jpg, tensor([[0.4967, 0.4896, 0.0293, 0.3056]]), tensor([[1]])
1127_target_06b7e77a9cd5b65c7fc71a

1181_stray_fb48db77b3410fd469b658a62bdb942a.jpg, tensor([[0.2036, 0.0399, 0.1010, 0.0799]]), tensor([[0]])
1182_target_0f3a86b10d5daf0b6820e0292b316b23.jpg, tensor([[0.4951, 0.8490, 0.0195, 0.3021]]), tensor([[1]])
1182_target_0f3a86b10d5daf0b6820e0292b316b23.jpg, tensor([[0.4951, 0.8490, 0.0195, 0.3021]]), tensor([[1]])
1183_stray_a1b06eceb0390c558a4f080964198fc8.jpg, tensor([[0.9853, 0.0885, 0.0293, 0.1771]]), tensor([[0]])
1183_stray_a1b06eceb0390c558a4f080964198fc8.jpg, tensor([[0.9853, 0.0885, 0.0293, 0.1771]]), tensor([[0]])
1185_stray_10aef41b25e9d5d7dda412703b2b6cd4.jpg, tensor([[0.3632, 0.7795, 0.0163, 0.1146]]), tensor([[0]])
1185_stray_10aef41b25e9d5d7dda412703b2b6cd4.jpg, tensor([[0.3632, 0.2205, 0.0163, 0.1146]]), tensor([[0]])
1186_target_de0602efe9521e9ea7f0bf4ee1dfa0d7.jpg, tensor([[0.2720, 0.9288, 0.0423, 0.1424]]), tensor([[0]])
1186_target_de0602efe9521e9ea7f0bf4ee1dfa0d7.jpg, tensor([[0.2720, 0.0712, 0.0423, 0.1424]]), tensor([[0]])
1188_target_805a662c92ddb5112affd

1238_target_c31980630fc3f82e723dc1d7c90c0893.jpg, tensor([[0.4935, 0.5052, 0.0293, 0.2465]]), tensor([[1]])
1239_target_481c2ce28e071182d332e3f603843ca8.jpg, tensor([[0.1987, 0.1198, 0.0326, 0.2396]]), tensor([[1]])
1239_target_481c2ce28e071182d332e3f603843ca8.jpg, tensor([[0.1987, 0.1198, 0.0326, 0.2396]]), tensor([[1]])
123_target_515dd3e5ced1ca6ee7223e092e8078c3.jpg, tensor([[0.4511, 0.8160, 0.0293, 0.2083]]), tensor([[1]])
123_target_515dd3e5ced1ca6ee7223e092e8078c3.jpg, tensor([[0.4511, 0.1840, 0.0293, 0.2083]]), tensor([[1]])
1241_target_7497b53badf5ecb381eb92c286cb5adf.jpg, tensor([[0.9495, 0.8993, 0.0293, 0.2014]]), tensor([[0]])
1241_target_7497b53badf5ecb381eb92c286cb5adf.jpg, tensor([[0.9495, 0.8993, 0.0293, 0.2014]]), tensor([[0]])
1242_target_26fac98cf5593142454d647de836e280.jpg, tensor([[0.5293, 0.5069, 0.0228, 0.2778]]), tensor([[1]])
1242_target_26fac98cf5593142454d647de836e280.jpg, tensor([[0.4707, 0.5069, 0.0228, 0.2778]]), tensor([[1]])
1243_stray_8342681936b6e3d0616

...

2645_target_95a4eeb590a844950cd3d44d0c916fce.jpg, tensor([[0.5668, 0.2240, 0.0326, 0.1840]]), tensor([[1]])
2647_stray_9e8dc23d131f978d850d107dc0222380.jpg, tensor([[0.3534, 0.8611, 0.0358, 0.0417]]), tensor([[0]])
2647_stray_9e8dc23d131f978d850d107dc0222380.jpg, tensor([[0.6466, 0.8611, 0.0358, 0.0417]]), tensor([[0]])
2650_target_447ef7fd895c6d32826dd2742cee6601.jpg, tensor([[0.5049, 0.5191, 0.0261, 0.2535]]), tensor([[1]])
2650_target_447ef7fd895c6d32826dd2742cee6601.jpg, tensor([[0.4951, 0.4809, 0.0261, 0.2535]]), tensor([[1]])
2652_target_d030d041293b7d9f2eb5b959687988d5.jpg, tensor([[0.4886, 0.5104, 0.0261, 0.2500]]), tensor([[1]])
2652_target_d030d041293b7d9f2eb5b959687988d5.jpg, tensor([[0.5114, 0.4896, 0.0261, 0.2500]]), tensor([[1]])
2656_target_7e4bfab1dbd98fb78996e0744cda7803.jpg, tensor([[0.4446, 0.5191, 0.0228, 0.2743]]), tensor([[1]])
2656_target_7e4bfab1dbd98fb78996e0744cda7803.jpg, tensor([[0.5554, 0.5191, 0.0228, 0.2743]]), tensor([[1]])
2658_stray_0efe71f670f967951eb

354_stray_b36386017dcbafcf8fb52e0abb9ef16c.jpg, tensor([[0.9821, 0.1059, 0.0358, 0.2118]]), tensor([[0]])
354_stray_b36386017dcbafcf8fb52e0abb9ef16c.jpg, tensor([[0.9821, 0.8941, 0.0358, 0.2118]]), tensor([[0]])
359_stray_6330cfeaeabde1a01c56c034946c614e.jpg, tensor([[0.4267, 0.5052, 0.0261, 0.1979]]), tensor([[0]])
359_stray_6330cfeaeabde1a01c56c034946c614e.jpg, tensor([[0.4267, 0.4948, 0.0261, 0.1979]]), tensor([[0]])
35_target_393df31022b0ddc78537bfbc36606083.jpg, tensor([[0.4902, 0.4878, 0.0293, 0.2118]]), tensor([[1]])
35_target_393df31022b0ddc78537bfbc36606083.jpg, tensor([[0.4902, 0.5122, 0.0293, 0.2118]]), tensor([[1]])
371_stray_d41ed289938c7c9cb5aa9818d2177768.jpg, tensor([[0.9674, 0.1597, 0.0651, 0.2222]]), tensor([[0]])
371_stray_d41ed289938c7c9cb5aa9818d2177768.jpg, tensor([[0.9674, 0.8403, 0.0651, 0.2222]]), tensor([[0]])
378_stray_c1d3d14b82a51acbdd7cad2e0a244da5.jpg, tensor([[0.4951, 0.5069, 0.0391, 0.3472]]), tensor([[0]])
378_stray_c1d3d14b82a51acbdd7cad2e0a244da5.jpg

587_stray_ee4b2539aa6e237d866c4331b458e21a.jpg, tensor([[0.6710, 0.7604, 0.0717, 0.1528]]), tensor([[0]])
58_stray_6da4063ead5757ee445cc91d4474b173.jpg, tensor([[0.4935, 0.4705, 0.0358, 0.2812]]), tensor([[0]])
58_stray_6da4063ead5757ee445cc91d4474b173.jpg, tensor([[0.4935, 0.5295, 0.0358, 0.2812]]), tensor([[0]])
591_target_7246cbc9397fa00d6c73540701feb8a1.jpg, tensor([[0.5147, 0.4948, 0.0261, 0.3229]]), tensor([[1]])
591_target_7246cbc9397fa00d6c73540701feb8a1.jpg, tensor([[0.4853, 0.4948, 0.0261, 0.3229]]), tensor([[1]])
593_target_7f8ec7152fa780c4d0ed4173663c4932.jpg, tensor([[0.4870, 0.5017, 0.0293, 0.2535]]), tensor([[1]])
593_target_7f8ec7152fa780c4d0ed4173663c4932.jpg, tensor([[0.4870, 0.5017, 0.0293, 0.2535]]), tensor([[1]])
594_target_8642255d900af9634f6884e16d0e0246.jpg, tensor([[0.4967, 0.5347, 0.0358, 0.2431]]), tensor([[1]])
594_target_8642255d900af9634f6884e16d0e0246.jpg, tensor([[0.4967, 0.4653, 0.0358, 0.2431]]), tensor([[1]])
596_target_dd5c9f45d68f5c6f5d24049bae464e5

751_target_820b76b58fa4effa67a090fbbc8bf993.jpg, tensor([[0.5000, 0.4774, 0.0293, 0.2257]]), tensor([[1]])
75_stray_9a370c9db5910bbd292851f77686be82.jpg, tensor([[0.2606, 0.9271, 0.0521, 0.1458]]), tensor([[0]])
75_stray_9a370c9db5910bbd292851f77686be82.jpg, tensor([[0.2606, 0.0729, 0.0521, 0.1458]]), tensor([[0]])
783_stray_be35d12123103af3706d6a8dce603f36.jpg, tensor([[0.3176, 0.3507, 0.0554, 0.1806]]), tensor([[0]])
783_stray_be35d12123103af3706d6a8dce603f36.jpg, tensor([[0.6824, 0.6493, 0.0554, 0.1806]]), tensor([[0]])
784_target_f4d261f84374aa997003c2c4d3edb9eb.jpg, tensor([[0.4967, 0.5139, 0.0228, 0.2500]]), tensor([[1]])
784_target_f4d261f84374aa997003c2c4d3edb9eb.jpg, tensor([[0.5033, 0.4861, 0.0228, 0.2500]]), tensor([[1]])
788_target_240caeceb028f324c71b0876bcd41b28.jpg, tensor([[0.5342, 0.4931, 0.0261, 0.2708]]), tensor([[1]])
788_target_240caeceb028f324c71b0876bcd41b28.jpg, tensor([[0.4658, 0.4931, 0.0261, 0.2708]]), tensor([[1]])
78_stray_fae7c2e03e099cb0347caada910323f1.j

979_stray_4926b4f2b3dd9f75db562aa31bec7c7f.jpg, tensor([[0.2932, 0.1875, 0.0261, 0.1458]]), tensor([[0]])
97_target_8d201ae70d426bda6d56d26ae75924e7.jpg, tensor([[0.4935, 0.5156, 0.0293, 0.2882]]), tensor([[1]])
97_target_8d201ae70d426bda6d56d26ae75924e7.jpg, tensor([[0.4935, 0.4844, 0.0293, 0.2882]]), tensor([[1]])
982_target_007341b58bbb65f2ff68bf8d0b55ba7e.jpg, tensor([[0.4625, 0.4948, 0.0391, 0.3090]]), tensor([[1]])
982_target_007341b58bbb65f2ff68bf8d0b55ba7e.jpg, tensor([[0.5375, 0.5052, 0.0391, 0.3090]]), tensor([[1]])
988_stray_1d3a7006d86ff8e1deb7a6401789a3ef.jpg, tensor([[0.4935, 0.3854, 0.0423, 0.2917]]), tensor([[0]])
988_stray_1d3a7006d86ff8e1deb7a6401789a3ef.jpg, tensor([[0.4935, 0.3854, 0.0423, 0.2917]]), tensor([[0]])
98_target_998e94ef1e668a10bac79c74803dc996.jpg, tensor([[0.5065, 0.5260, 0.0423, 0.2951]]), tensor([[1]])
98_target_998e94ef1e668a10bac79c74803dc996.jpg, tensor([[0.4935, 0.4740, 0.0423, 0.2951]]), tensor([[1]])
992_stray_5cb5c4fd1fb823ad1e8b2e177e1049c8.j

In [14]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="yolo_swin_RADIO",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "batch_size": BATCH_SIZE,
    "architecture": BACKBONE,
    "dataset": PART,
    "epochs": num_epochs,
    "patch factor":PATCH_FACTOR,
    "aug factor":AUG_FACTOR,
    }
)

wandb: Currently logged in as: gomduribo (urp). Use `wandb login --relogin` to force relogin


In [15]:
best_epoch = 0
best_score = float('inf')
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    train_loss, val_loss = train_one_epoch(dataloaders, model, criterion, optimizer, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
#     train_loss["obj_loss"] += obj_loss
#     train_loss["noobj_loss"] += noobj_loss
#     train_loss["bbox_loss"] += bbox_loss
#     train_loss["cls_loss"] += cls_loss
    wandb.log({"Train Loss": train_loss['total_loss'],
               "Train obj Loss":train_loss["obj_loss"],
               "Train bbox Loss":train_loss["bbox_loss"],
               "Train class Loss":train_loss["cls_loss"],
               "Val Loss": val_loss['total_loss'],
               "Val obj Loss":val_loss["obj_loss"],
               "Val bbox Loss":val_loss["bbox_loss"],
               "Val class Loss":val_loss["cls_loss"],})
    print(f"\nepoch:{epoch+1}/{num_epochs} - Train Loss: {train_loss['total_loss']:.4f}, Val Loss: {val_loss['total_loss']:.4f}\n")
    
    if (epoch+1) % 10 == 0:
        save_model(model.state_dict(), f'model_{epoch+1}.pth', save_dir=f"./trained_model/{BACKBONE}_{PART}_LR{LR}_IP{PATCH_FACTOR}_AUG{AUG_FACTOR}")
wandb.finish()

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3423.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<<<iteration:[20/244] - total_loss: 7.4863  obj_loss: 0.0440  noobj_loss: 3.4154  bbox_loss: 1.0108  cls_loss: 0.6805  
<<<iteration:[40/244] - total_loss: 3.0774  obj_loss: 0.0300  noobj_loss: 2.5132  bbox_loss: 0.2922  cls_loss: 0.3296  
<<<iteration:[60/244] - total_loss: 3.1319  obj_loss: 0.0247  noobj_loss: 2.3742  bbox_loss: 0.3162  cls_loss: 0.3390  
<<<iteration:[80/244] - total_loss: 2.8327  obj_loss: 0.0380  noobj_loss: 2.3271  bbox_loss: 0.2696  cls_loss: 0.2830  
<<<iteration:[100/244] - total_loss: 2.6597  obj_loss: 0.0231  noobj_loss: 1.9910  bbox_loss: 0.2712  cls_loss: 0.2849  
<<<iteration:[120/244] - total_loss: 2.1812  obj_loss: 0.0208  noobj_loss: 1.6983  bbox_loss: 0.2062  cls_loss: 0.2801  
<<<iteration:[140/244] - total_loss: 1.9218  obj_loss: 0.0185  noobj_loss: 1.5139  bbox_loss: 0.1748  cls_loss: 0.2725  
<<<iteration:[160/244] - total_loss: 1.8527  obj_loss: 0.0212  noobj_loss: 1.4022  bbox_loss: 0.1713  cls_loss: 0.2740  
<<<iteration:[180/244] - total_loss:

KeyboardInterrupt: 

# Test Dataset Inference

In [13]:
import numpy as np
import os 
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torch import nn
import torchsummary
from torch.utils.data import DataLoader
from collections import defaultdict
from torchvision.utils import make_grid

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [39]:
class YOLO_RESNET18(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.num_classes = num_classes
        self.num_bboxes = 2
        self.grid_size = 7

        resnet18 = torchvision.models.resnet18(pretrained = True)
#         swin=torchvision.models.swin_v2_t(weights='IMAGENET1K_V1')
        layers = [m for m in resnet18.children()] #Resnet에서 Yolo에서 가져올수 있을만한 layer만 선별적으로 가져오기 위해서

        # 기존 Resnet18의 layer들중에서 맨 뒤에 두개만 제외하고 다 가져와서 Backbone으로 사용
        self.backbone = nn.Sequential(*layers[:-2]) 
        self.head = nn.Sequential(
                nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=1, padding=0,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),

                nn.Conv2d(in_channels=1024, out_channels=(4+1)*self.num_bboxes+num_classes, kernel_size=1, padding=0, bias=False),
                nn.AdaptiveAvgPool2d(output_size=(self.grid_size, self.grid_size))
            )

    def forward(self, x):
        out = self.backbone(x)
        # out = self.neck(out)
        out = self.head(out) # input (batch, 3, 448, 448) -> output feature (batch, 12, 7, 7)
        return out

In [40]:
def load_model(ckpt_path, num_classes, device):
    checkpoint = torch.load(ckpt_path, map_location=device)
#     model = YOLO_SWIN(num_classes=num_classes)
    model=YOLO_RESNET18(num_classes=num_classes)
    model.load_state_dict(checkpoint)
    model = model.to(device)
    model.eval()
    return model

In [41]:
IMAGE_SIZE=448
transformer = A.Compose([
            A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
            A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
)

In [43]:
NUM_CLASSES=2
# ckpt_path="./trained_model/YOLO_SWIN_T_body_LR0.0001_AUG30/model_90.pth"
ckpt_path="/workspace/Plastic_Bottle_defect_detection/trained_model/YOLO_RESNET18_neck_LR0.0001_AUG20//model_100.pth"
model = load_model(ckpt_path, NUM_CLASSES, device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
NECK_PATH = '/home/host_data/PET_data/Neck'
BODY_PATH = '/home/host_data/PET_data/Body'
test_dataset=PET_dataset("neck" ,neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='test', transformer=transformer, aug=None)
test_dataloaders = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

start making augmented images-- augmented factor:0
total length of augmented images: 0


In [45]:
len(test_dataset)

25

In [46]:
@torch.no_grad()
def model_predict(image, model, conf_thres=0.2, iou_threshold=0.1):
    predictions = model(image)
    prediction = predictions.detach().cpu().squeeze(dim=0)
    f_map=prediction

#     print(prediction.shape)
    
    grid_size = prediction.shape[-1]
    y_grid, x_grid = torch.meshgrid(torch.arange(grid_size), torch.arange(grid_size))
    stride_size = IMAGE_SIZE/grid_size

    conf = prediction[[0,5], ...].reshape(1, -1)
    xc = (prediction[[1,6], ...] * IMAGE_SIZE + x_grid*stride_size).reshape(1,-1)
    yc = (prediction[[2,7], ...] * IMAGE_SIZE + y_grid*stride_size).reshape(1,-1)
    w = (prediction[[3,8], ...] * IMAGE_SIZE).reshape(1,-1)
    h = (prediction[[4,9], ...] * IMAGE_SIZE).reshape(1,-1)
    cls = torch.max(prediction[10:, ...].reshape(NUM_CLASSES, -1), dim=0).indices.tile(1,2)
    
    x_min = xc - w/2
    y_min = yc - h/2
    x_max = xc + w/2
    y_max = yc + h/2

    prediction_res = torch.cat([x_min, y_min, x_max, y_max, conf, cls], dim=0)
    prediction_res = prediction_res.transpose(0,1)

    # x_min과 y_min이 음수가 되지않고, x_max와 y_max가 이미지 크기를 넘지 않게 제한
    prediction_res[:, 2].clip(min=0, max=image.shape[1]) 
    prediction_res[:, 3].clip(min=0, max=image.shape[0])
        
    pred_res = prediction_res[prediction_res[:, 4] > conf_thres]
    nms_index = torchvision.ops.nms(boxes=pred_res[:, 0:4], scores=pred_res[:, 4], iou_threshold=iou_threshold)
    pred_res_ = pred_res[nms_index].numpy()
    
    n_obj = pred_res_.shape[0]
    bboxes = np.zeros(shape=(n_obj, 4), dtype=np.float32)
    bboxes[:, 0:2] = (pred_res_[:, 0:2] + pred_res_[:, 2:4]) / 2
    bboxes[:, 2:4] = pred_res_[:, 2:4] - pred_res_[:, 0:2]
    scores = pred_res_[:, 4]
    class_ids = pred_res_[:, 5]
    
    # 이미지 값이 들어가면 모델을 통해서, 후처리까지 포함된 yolo 포멧의 box좌표, 그 좌표에 대한 confidence score
    # 그리고 class id를 반환
    return bboxes, scores, class_ids,f_map

In [47]:
pred_images = []
pred_labels =[]
feature_maps=[]

for index, batch in enumerate(test_dataloaders):
    images = batch[0].to(device)
    bboxes, scores, class_ids, fmap = model_predict(images, model, conf_thres=0.1, iou_threshold=0.1)
    
    if len(bboxes) > 0:
        prediction_yolo = np.concatenate([bboxes, scores[:, np.newaxis], class_ids[:, np.newaxis]], axis=1)
    else:
        prediction_yolo = np.array([])
    
    # 텐서형의 이미지를 다시 unnormalize를 시키고, 다시 chw를 hwc로 바꾸고 넘파이로 바꾼다.
    np_image = make_grid(images[0], normalize=True).cpu().permute(1,2,0).numpy()
    pred_images.append(np_image)
    pred_labels.append(prediction_yolo)
    feature_maps.append(fmap)

    

In [48]:
from ipywidgets import interact

@interact(index=(0,len(pred_images)-1))
def show_result(index=0):
    print(pred_labels[index])
    if len(pred_labels[index]) > 0:
        result = visualize(pred_images[index], pred_labels[index][:, 0:4], pred_labels[index][:, 5])
    else:
        result = pred_images[index]
        
    plt.figure(figsize=(6,6))
    plt.imshow(result)
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=24), Output()), _dom_classes=('widget-intera…

In [49]:
#feature map에서 0,5번쨰에 해당하는 objectness 투사
from ipywidgets import interact

@interact(index=(0,len(pred_images)-1))
def show_result(index=0):
    print(pred_labels[index])
    if len(pred_labels[index]) > 0:
        result = visualize(pred_images[index], pred_labels[index][:, 0:4], pred_labels[index][:, 5])
    else:
        result = pred_images[index]
    
    f_map=feature_maps[index]
    zero_canvas=np.zeros((448,448))

    cv_re1=cv2.resize(f_map[0,:,:].numpy(),(448,448))
    cv_re2=cv2.resize(f_map[5,:,:].numpy(),(448,448))
    zero_canvas=zero_canvas+cv_re1+cv_re2

    
    fig = plt.figure()
    rows = 1
    cols = 2
    ax1 = fig.add_subplot(rows, cols, 1)
    ax1.imshow(result)
    ax1.set_title('Detection')
    ax1.axis("off")
    
    ax2 = fig.add_subplot(rows, cols, 2)
    ax2.imshow(zero_canvas)
    ax2.set_title('feature map-objectness')
    ax2.axis("off")

    plt.show()


interactive(children=(IntSlider(value=0, description='index', max=24), Output()), _dom_classes=('widget-intera…